Udacity deep learning [assignment 2](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/udacity/2_fullyconnected.ipynb)

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


### Logistic Regression using tf

In [4]:
batch_size = 512

In [ ]:
graph_LR = tf.Graph()

with graph_LR.as_default():
#     tf_train_dataset = tf.constant(train_dataset)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    x = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    with tf.variable_scope("LogisticRegression"):
        W = tf.get_variable("weights", (image_size*image_size,num_labels),initializer=tf.random_uniform_initializer())
        b = tf.get_variable("bias",(num_labels,), initializer=tf.constant_initializer(0.0))
        biases = tf.Variable(tf.zeros([num_labels]))
        print(x.get_shape(),W.get_shape(),b.get_shape(), biases.get_shape())
        logits = tf.matmul(x, W) + b
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
        optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
#         train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, W) + b)
        valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, W) + b)
        test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, W) + b)

In [5]:
def accuracy(pred, labels):
    return (100.0 * np.sum(np.argmax(pred,axis=1) == np.argmax(labels,axis=1)) / pred.shape[0])

In [9]:
num_steps = 2001

In [ ]:
%%timeit -r1 -n1
with tf.Session(graph=graph_LR) as sess:
    sess.run(tf.initialize_all_variables())
    print("Initialized .")
    for i in xrange(num_steps):
        indices = np.random.choice(train_dataset.shape[0], batch_size)
        x_batch = train_dataset[indices]
        y_batch = train_labels[indices]
        _, l = sess.run([optimizer, loss], feed_dict={x:x_batch, y:y_batch})
        if i % 100 == 0:
            print("Loss at step %i = %.5f" %(i, l))
            #t_pred = train_prediction.eval()
            v_pred = valid_prediction.eval()
#             print("Training accuracy = %.2f, Validation accuracy = %.2f" \
#                   %(accuracy(t_pred,train_labels), accuracy(v_pred, valid_labels)))
            print("Validation accuracy = %.2f" %(accuracy(v_pred, valid_labels)))
    print("Completed training .")
    test_pred = test_prediction.eval()
    print("Test accuracy = %.2f" % accuracy(test_pred, test_labels))
        

### Hidden layer

In [15]:
graph_H1 = tf.Graph()
hidden_layers = 1000
with graph_H1.as_default():
    with tf.device('/gpu:0'):
    #     tf_train_dataset = tf.constant(train_dataset)
        tf_valid_dataset = tf.constant(valid_dataset)
        tf_test_dataset = tf.constant(test_dataset)
        x = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
        y = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        with tf.variable_scope("1HiddenLayer"):
            W1 = tf.get_variable("weights_layer_1", \
                                 (image_size*image_size, hidden_layers),\
                                 initializer=tf.random_uniform_initializer())
            b1 = tf.get_variable("bias_layer_1",\
                                 (hidden_layers,), \
                                 initializer=tf.constant_initializer(0.0))
            print(x.get_shape(),W1.get_shape(),b1.get_shape())
            relu_layer_1 = tf.nn.relu((tf.matmul(x, W1) + b1),name="relu_layer_1")
            W2 = tf.get_variable("weights_layer_hidden", \
                                 (hidden_layers, num_labels), \
                                 initializer=tf.random_uniform_initializer())
            b2 = tf.get_variable("bias_layer_hidden", (num_labels,),\
                                initializer=tf.constant_initializer(0.0))
            print(relu_layer_1.get_shape(),W2.get_shape(),b2.get_shape())
            logits = tf.matmul(relu_layer_1, W2) + b2

            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y))
            optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    #         train_prediction = tf.nn.softmax(tf.matmul(tf_train_dataset, W) + b)
            valid_prediction = tf.nn.softmax(\
                                             tf.matmul(\
                                                       tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1),\
                                                      W2)+b2)
            test_prediction = tf.nn.softmax(\
                                             tf.matmul(\
                                                       tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1),\
                                                      W2)+b2)

(512, 784) (784, 1000) (1000,)
(512, 1000) (1000, 10) (10,)


In [16]:
%%timeit -r1 -n1
with tf.Session(graph=graph_H1, config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(tf.initialize_all_variables())
    print("Initialized .")
    for i in xrange(num_steps):
        indices = np.random.choice(train_dataset.shape[0], batch_size)
        x_batch = train_dataset[indices]
        y_batch = train_labels[indices]
        _, l = sess.run([optimizer, loss], feed_dict={x:x_batch, y:y_batch})
        if i % 100 == 0:
            print("Loss at step %i = %.5f" %(i, l))
            #t_pred = train_prediction.eval()
            v_pred = valid_prediction.eval()
#             print("Training accuracy = %.2f, Validation accuracy = %.2f" \
#                   %(accuracy(t_pred,train_labels), accuracy(v_pred, valid_labels)))
            print("Validation accuracy = %.2f" %(accuracy(v_pred, valid_labels)))
    print("Completed training .")
    test_pred = test_prediction.eval()
    print("Test accuracy = %.2f" % accuracy(test_pred, test_labels))
        

Initialized .
Loss at step 0 = 211.37839
Validation accuracy = 11.69
Loss at step 100 = 38.32451
Validation accuracy = 23.04
Loss at step 200 = 163.38474
Validation accuracy = 29.75
Loss at step 300 = 6.04366
Validation accuracy = 33.74
Loss at step 400 = 2.76500
Validation accuracy = 37.68
Loss at step 500 = 2.33596
Validation accuracy = 39.04
Loss at step 600 = 14.74561
Validation accuracy = 41.10
Loss at step 700 = 10.60199
Validation accuracy = 47.15
Loss at step 800 = 1.87231
Validation accuracy = 48.39
Loss at step 900 = 1.56463
Validation accuracy = 52.56
Loss at step 1000 = 1.38468
Validation accuracy = 51.97
Loss at step 1100 = 1.57524
Validation accuracy = 57.13
Loss at step 1200 = 1.18497
Validation accuracy = 56.46
Loss at step 1300 = 1.13007
Validation accuracy = 59.19
Loss at step 1400 = 1.24248
Validation accuracy = 67.84
Loss at step 1500 = 0.85601
Validation accuracy = 71.49
Loss at step 1600 = 1.20706
Validation accuracy = 72.92
Loss at step 1700 = 1.20432
Validation 